In [1]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from matplotlib import pyplot as plt
import statsmodels.api as sm

In [2]:
import import_ipynb
from evaluation import precision
from runTime import runTime

importing Jupyter notebook from evaluation.ipynb
importing Jupyter notebook from runTime.ipynb


In [3]:
from evaluation2 import precision2

importing Jupyter notebook from evaluation2.ipynb


In [4]:
df1=pd.read_csv("E:\\5th Sem\\DMA Project\\Model Evaluation\\weibo_train1_cp.csv")
df2=pd.read_csv("E:\\5th Sem\\DMA Project\\Model Evaluation\\weibo_train2_cp.csv")
frames=[df1,df2]
train_dataset=pd.concat(frames)
predict_dataset=pd.read_csv("E:\\5th Sem\\DMA Project\\Model Evaluation\\weibo_predict_cp.csv")

In [5]:
train_dataset['date']=pd.to_datetime(train_dataset['date'],errors='coerce')

In [6]:
train_month=[g for n, g in train_dataset.groupby(pd.Grouper(key='date',freq='M'))]

In [7]:
train_dataset['time']=pd.to_datetime(train_dataset['time'],errors='coerce')

In [8]:
train_hour=[g for n, g in train_dataset.groupby(pd.Grouper(key='time',freq='H'))]

In [9]:
train_month[0].to_csv("E:\\5th Sem\\DMA Project\\Model Evaluation\\weibo_train_feb_cp.csv",sep=',',index=False,encoding='utf-8')
train_month[1].to_csv("E:\\5th Sem\\DMA Project\\Model Evaluation\\weibo_train_march_cp.csv",sep=',',index=False,encoding='utf-8')
train_month[2].to_csv("E:\\5th Sem\\DMA Project\\Model Evaluation\\weibo_train_april_cp.csv",sep=',',index=False,encoding='utf-8')
train_month[3].to_csv("E:\\5th Sem\\DMA Project\\Model Evaluation\\weibo_train_may_cp.csv",sep=',',index=False,encoding='utf-8')
train_month[4].to_csv("E:\\5th Sem\\DMA Project\\Model Evaluation\\weibo_train_june_cp.csv",sep=',',index=False,encoding='utf-8')
train_month[5].to_csv("E:\\5th Sem\\DMA Project\\Model Evaluation\\weibo_train_july_cp.csv",sep=',',index=False,encoding='utf-8')

In [13]:
i=0
for i in range(0,24):
    path="E:\\5th Sem\\DMA Project\\Model Evaluation\\weibo_train_hour_"+str(i)+"_cp.csv"
    train_hour[i].to_csv(path,sep=',',index=False,encoding='utf-8')


In [14]:
frames1=[train_month[0],train_month[1],train_month[2],train_month[3],train_month[4]]
train=pd.concat(frames1)
predict=train_month[5]

# Model 1 (Factors: Media, #, @, ?, !, Length, Emoji)

In [15]:
X_train=train[["content_media_count","content_#_count","content_@_count","content_?_count","content_!_count","content_length","content_emoji_count"]]
Y_train=train[["forward_count","comment_count","like_count"]]
X_test=predict[["content_media_count","content_#_count","content_@_count","content_?_count","content_!_count","content_length","content_emoji_count"]]
Y_test=predict[["forward_count","comment_count","like_count"]]

In [16]:
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(1044681, 7) (1044681, 3)
(184937, 7) (184937, 3)


In [17]:
pd.options.mode.use_inf_as_na = True
X_train.fillna(X_train.max(),inplace=True)
X_test.fillna(X_test.max(),inplace=True)

C:\Users\DELL\Anaconda3\lib\site-packages\pandas\core\generic.py:5430: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [18]:
lm=linear_model.LinearRegression()
model=lm.fit(X_train,Y_train)
pred=lm.predict(X_test)
pred=pred.round()
pred=(np.maximum(pred,0.))

In [19]:
print(pred[0:5])

[[ 4.  1.  2.]
 [ 3.  2.  3.]
 [ 3.  1.  1.]
 [ 7.  1.  2.]
 [ 9.  2.  3.]]


In [20]:
print(model.coef_)

[[-2.13937044 -0.36595303 -0.29844433 -0.04270781 -0.20884535  0.05323583
  -0.11373958]
 [-1.46356558 -0.15223682 -0.02946097 -0.00326344 -0.04880305  0.00769467
   0.1747673 ]
 [-2.6330917  -0.13749083 -0.32811723 -0.01394959 -0.071054    0.01544652
   0.26583545]]


In [21]:
print(model.intercept_)

[ 1.31437859  1.76293348  2.98461306]


In [22]:
np.savetxt("E:\\5th Sem\\DMA Project\\Model Evaluation\\weibo_predict_result2.csv",pred,delimiter=',',header="forward_count,comment_count,like_count",comments="")
result=pd.read_csv("E:\\5th Sem\\DMA Project\\Model Evaluation\\weibo_predict_result2.csv")

In [23]:
train_real_pred = Y_test
forward=result['forward_count'].values
comment=result['forward_count'].values
like=result['forward_count'].values
train_real_pred['fp'],train_real_pred['cp'],train_real_pred['lp'] = forward,comment,like
print ("Score on the training set:{0:.2f}%".format(precision(train_real_pred.values)*100))

C:\Users\DELL\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


Score on the training set:3.69%


# Model 2 (Media, Length, Emoji)

In [27]:
X_train=train[["content_media_count","content_length","content_emoji_count"]]
Y_train=train[["forward_count","comment_count","like_count"]]
X_test=predict[["content_media_count","content_length","content_emoji_count"]]
Y_test=predict[["forward_count","comment_count","like_count"]]

In [28]:
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(1044681, 3) (1044681, 3)
(184937, 3) (184937, 3)


In [29]:
pd.options.mode.use_inf_as_na = True
X_train.fillna(X_train.max(),inplace=True)
X_test.fillna(X_test.max(),inplace=True)

C:\Users\DELL\Anaconda3\lib\site-packages\pandas\core\generic.py:5430: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [30]:
lm=linear_model.LinearRegression()
model=lm.fit(X_train,Y_train)
pred=lm.predict(X_test)
pred=pred.round()
pred=(np.maximum(pred,0.))

In [31]:
print(pred[0:5])

[[ 4.  1.  2.]
 [ 3.  2.  3.]
 [ 3.  1.  1.]
 [ 7.  1.  2.]
 [ 8.  1.  3.]]


In [32]:
print(model.coef_)

[[-2.19780299  0.05049652 -0.14777505]
 [-1.47377848  0.00679751  0.16018448]
 [-2.68756793  0.01390734  0.25398113]]


In [33]:
print(model.intercept_)

[ 1.23425639  1.73473692  2.94322868]


In [35]:
np.savetxt("E:\\5th Sem\\DMA Project\\Model Evaluation\\weibo_predict_result3.csv",pred,delimiter=',',header="forward_count,comment_count,like_count",comments="")
result=pd.read_csv("E:\\5th Sem\\DMA Project\\Model Evaluation\\weibo_predict_result3.csv")

In [36]:
train_real_pred = Y_test
forward=result['forward_count'].values
comment=result['forward_count'].values
like=result['forward_count'].values
train_real_pred['fp'],train_real_pred['cp'],train_real_pred['lp'] = forward,comment,like
print ("Score on the training set:{0:.2f}%".format(precision(train_real_pred.values)*100))

C:\Users\DELL\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


Score on the training set:3.79%


# Model 3(Media)

In [37]:
X_train=train[["content_media_count"]]
Y_train=train[["forward_count","comment_count","like_count"]]
X_test=predict[["content_media_count"]]
Y_test=predict[["forward_count","comment_count","like_count"]]

In [38]:
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(1044681, 1) (1044681, 3)
(184937, 1) (184937, 3)


In [39]:
pd.options.mode.use_inf_as_na = True
X_train.fillna(X_train.max(),inplace=True)
X_test.fillna(X_test.max(),inplace=True)

C:\Users\DELL\Anaconda3\lib\site-packages\pandas\core\generic.py:5430: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [40]:
lm=linear_model.LinearRegression()
model=lm.fit(X_train,Y_train)
pred=lm.predict(X_test)
pred=pred.round()
pred=(np.maximum(pred,0.))

In [41]:
print(pred[0:5])

[[ 3.  1.  1.]
 [ 4.  2.  4.]
 [ 3.  1.  1.]
 [ 3.  1.  1.]
 [ 3.  1.  1.]]


In [42]:
print(model.coef_)

[[-0.62388914]
 [-1.26343548]
 [-2.25659216]]


In [43]:
print(model.intercept_)

[ 3.86255173  2.09158813  3.67207903]


In [45]:
np.savetxt("E:\\5th Sem\\DMA Project\\Model Evaluation\\weibo_predict_result4.csv",pred,delimiter=',',header="forward_count,comment_count,like_count",comments="")
result=pd.read_csv("E:\\5th Sem\\DMA Project\\Model Evaluation\\weibo_predict_result4.csv")

In [33]:
train_real_pred = Y_test
train_real_pred['fp']=result['forward_count'].values
train_real_pred['cp']=result['comment_count'].values
train_real_pred['lp']=result['like_count'].values
train_real_pred=train_real_pred.round()
print ("Score on the training set:{0:.2f}%".format(precision(train_real_pred.values)*100))

G:\Anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
G:\Anaconda\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
G:\Anaconda\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.

Score on the training set:3.59%


# Model 4 (Time) Pre-requisite

In [34]:
train_dataset['hour']=pd.DatetimeIndex(train_dataset['time']).hour
print("done")
train_dataset['min']=pd.DatetimeIndex(train_dataset['time']).minute
print("done")
train_dataset['sec']=pd.DatetimeIndex(train_dataset['time']).second
print("done")

done
done
done


In [95]:
predict_dataset['hour']=pd.DatetimeIndex(predict_dataset['time']).hour
print("done")
predict_dataset['min']=pd.DatetimeIndex(predict_dataset['time']).minute
print("done")
predict_dataset['sec']=pd.DatetimeIndex(predict_dataset['time']).second
print("done")

done
done
done


In [98]:
train_dataset.head(614809).to_csv("G://DMA_PROJECT//weibo_train1_cpt.csv",sep=',',index=False,encoding='utf-8')
train_dataset.tail(614809).to_csv("G://DMA_PROJECT//weibo_train2_cpt.csv",sep=',',index=False,encoding='utf-8')
predict_dataset.to_csv("G://DMA_PROJECT//weibo_predict_cpt.csv",sep=',',index=False,encoding='utf-8')

In [4]:
df1=pd.read_csv("G://DMA_PROJECT//weibo_train1_cpt.csv")
df2=pd.read_csv("G://DMA_PROJECT//weibo_train2_cpt.csv")
frames=[df1,df2]
train_dataset=pd.concat(frames)
predict_dataset=pd.read_csv("G://DMA_PROJECT//weibo_predict_cpt.csv")

In [100]:
train_dataset['date']=pd.to_datetime(train_dataset['date'],errors='coerce')

In [101]:
train_month=[g for n, g in train_dataset.groupby(pd.Grouper(key='date',freq='M'))]

In [102]:
train_month[0].to_csv("G://DMA_PROJECT//weibo_train_feb_cpt.csv",sep=',',index=False,encoding='utf-8')
train_month[1].to_csv("G://DMA_PROJECT//weibo_train_march_cpt.csv",sep=',',index=False,encoding='utf-8')
train_month[2].to_csv("G://DMA_PROJECT//weibo_train_april_cpt.csv",sep=',',index=False,encoding='utf-8')
train_month[3].to_csv("G://DMA_PROJECT//weibo_train_may_cpt.csv",sep=',',index=False,encoding='utf-8')
train_month[4].to_csv("G://DMA_PROJECT//weibo_train_june_cpt.csv",sep=',',index=False,encoding='utf-8')
train_month[5].to_csv("G://DMA_PROJECT//weibo_train_july_cpt.csv",sep=',',index=False,encoding='utf-8')

In [35]:
train_month[0]=pd.read_csv("G://DMA_PROJECT//weibo_train_feb_cpt.csv")
train_month[1]=pd.read_csv("G://DMA_PROJECT//weibo_train_march_cpt.csv")
train_month[2]=pd.read_csv("G://DMA_PROJECT//weibo_train_april_cpt.csv")
train_month[3]=pd.read_csv("G://DMA_PROJECT//weibo_train_may_cpt.csv")
train_month[4]=pd.read_csv("G://DMA_PROJECT//weibo_train_june_cpt.csv")
train_month[5]=pd.read_csv("G://DMA_PROJECT//weibo_train_july_cpt.csv")

In [36]:
frames1=[train_month[0],train_month[1],train_month[2],train_month[3],train_month[4]]
train=pd.concat(frames1)
predict=train_month[5]

# Model 4 (Time: Hour, Min, Sec)

In [37]:
X_train=train[["hour","min","sec"]]
Y_train=train[["forward_count","comment_count","like_count"]]
X_test=predict[["hour","min","sec"]]
Y_test=predict[["forward_count","comment_count","like_count"]]

In [38]:
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(1044681, 3) (1044681, 3)
(184937, 3) (184937, 3)


In [39]:
pd.options.mode.use_inf_as_na = True
X_train.fillna(X_train.max(),inplace=True)
X_test.fillna(X_test.max(),inplace=True)

G:\Anaconda\lib\site-packages\pandas\core\generic.py:5430: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [40]:
lm=linear_model.LinearRegression()
model=lm.fit(X_train,Y_train)
pred=lm.predict(X_test)
pred=pred.round()
pred=(np.maximum(pred,0.))

In [41]:
print(pred[0:5])

[[ 3.  1.  2.]
 [ 2.  1.  2.]
 [ 3.  1.  2.]
 [ 1.  1.  1.]
 [ 6.  1.  3.]]


In [42]:
print(model.coef_)

[[ 0.08066778 -0.06019321 -0.0382752 ]
 [ 0.02746039 -0.00495496 -0.00478928]
 [ 0.07295777 -0.03270772 -0.01755154]]


In [43]:
print(model.intercept_)

[ 5.15832365  1.15751239  2.61598361]


In [44]:
np.savetxt("G://DMA_PROJECT//weibo_predict_result5.csv",pred,delimiter=',',header="forward_count,comment_count,like_count",comments="")
result=pd.read_csv("G://DMA_PROJECT//weibo_predict_result5.csv")

In [45]:
train_real_pred = Y_test
train_real_pred['fp']=result['forward_count'].values
train_real_pred['cp']=result['comment_count'].values
train_real_pred['lp']=result['like_count'].values
train_real_pred=train_real_pred.round()
print ("Score on the training set:{0:.2f}%".format(precision(train_real_pred.values)*100))

G:\Anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
G:\Anaconda\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
G:\Anaconda\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.

Score on the training set:4.19%


# Model 5 (Time: Hour)

In [46]:
X_train=train[["hour"]]
Y_train=train[["forward_count","comment_count","like_count"]]
X_test=predict[["hour"]]
Y_test=predict[["forward_count","comment_count","like_count"]]

In [47]:
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(1044681, 1) (1044681, 3)
(184937, 1) (184937, 3)


In [48]:
pd.options.mode.use_inf_as_na = True
X_train.fillna(X_train.max(),inplace=True)
X_test.fillna(X_test.max(),inplace=True)

G:\Anaconda\lib\site-packages\pandas\core\generic.py:5430: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [49]:
lm=linear_model.LinearRegression()
model=lm.fit(X_train,Y_train)
pred=lm.predict(X_test)
pred=pred.round()
pred=(np.maximum(pred,0.))

In [50]:
print(pred[0:5])

[[ 3.  1.  1.]
 [ 4.  1.  2.]
 [ 3.  1.  2.]
 [ 3.  1.  2.]
 [ 3.  1.  2.]]


In [51]:
print(model.coef_)

[[ 0.07590631]
 [ 0.02704577]
 [ 0.0704154 ]]


In [52]:
print(model.intercept_)

[ 2.40776056  0.88480607  1.21309106]


In [53]:
np.savetxt("G://DMA_PROJECT//weibo_predict_result6.csv",pred,delimiter=',',header="forward_count,comment_count,like_count",comments="")
result=pd.read_csv("G://DMA_PROJECT//weibo_predict_result6.csv")

In [54]:
train_real_pred = Y_test
train_real_pred['fp']=result['forward_count'].values
train_real_pred['cp']=result['comment_count'].values
train_real_pred['lp']=result['like_count'].values
train_real_pred=train_real_pred.round()
print ("Score on the training set:{0:.2f}%".format(precision(train_real_pred.values)*100))

G:\Anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
G:\Anaconda\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
G:\Anaconda\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.

Score on the training set:3.66%


# Model 6  Time: (Hour, Min, Sec), Media, Length, Emoji

In [55]:
X_train=train[["content_media_count","content_length","content_emoji_count","hour","min","sec"]]
Y_train=train[["forward_count","comment_count","like_count"]]
X_test=predict[["content_media_count","content_length","content_emoji_count","hour","min","sec"]]
Y_test=predict[["forward_count","comment_count","like_count"]]

In [56]:
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(1044681, 6) (1044681, 3)
(184937, 6) (184937, 3)


In [57]:
pd.options.mode.use_inf_as_na = True
X_train.fillna(X_train.max(),inplace=True)
X_test.fillna(X_test.max(),inplace=True)

G:\Anaconda\lib\site-packages\pandas\core\generic.py:5430: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [58]:
lm=linear_model.LinearRegression()
model=lm.fit(X_train,Y_train)
pred=lm.predict(X_test)
pred=pred.round()
pred=(np.maximum(pred,0.))

In [59]:
print(pred[0:5])

[[  4.   1.   1.]
 [  1.   2.   3.]
 [  2.   1.   1.]
 [  4.   1.   1.]
 [ 10.   2.   4.]]


In [60]:
print(model.coef_)

[[-2.16126691  0.04995623 -0.14079719  0.09097968 -0.05802816 -0.03386975]
 [-1.46714342  0.0068272   0.15966872  0.02589544 -0.00483482 -0.00418282]
 [-2.6652836   0.0137497   0.25624217  0.07054347 -0.03242168 -0.01633076]]


In [61]:
print(model.intercept_)

[ 2.62859063  1.63033587  3.36618167]


In [62]:
np.savetxt("G://DMA_PROJECT//weibo_predict_result7.csv",pred,delimiter=',',header="forward_count,comment_count,like_count",comments="")
result=pd.read_csv("G://DMA_PROJECT//weibo_predict_result7.csv")

In [63]:
train_real_pred = Y_test
train_real_pred['fp']=result['forward_count'].values
train_real_pred['cp']=result['comment_count'].values
train_real_pred['lp']=result['like_count'].values
train_real_pred=train_real_pred.round()
print ("Score on the training set:{0:.2f}%".format(precision(train_real_pred.values)*100))

G:\Anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
G:\Anaconda\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
G:\Anaconda\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.

Score on the training set:6.89%


# Model 7: Stats Pre-requisite

In [5]:
df1=pd.read_csv("G://DMA_PROJECT//weibo_train1_cpt.csv")
df2=pd.read_csv("G://DMA_PROJECT//weibo_train2_cpt.csv")
frames=[df1,df2]
train_dataset=pd.concat(frames)
predict_dataset=pd.read_csv("G://DMA_PROJECT//weibo_predict_cpt.csv")

In [6]:
stat=pd.read_csv("G://DMA_PROJECT//train_uid_stat.csv")

In [7]:
trainstat=pd.merge(train_dataset,stat,on=['u_id'])
predictstat=pd.merge(train_dataset,stat,on=['u_id'])

In [9]:
trainstat.head(5)

,u_id,m_id,forward_count,comment_count,like_count,content,date,time,content_media_count,content_#_count,...,forward_median,forward_mean,comment_min,comment_max,comment_median,comment_mean,like_min,like_max,like_median,like_mean
0,d38e9bed5d98110dc2489d0d1cac3c2a,7d45833d9865727a88b960b0603c19f6,0,0,0,丽江旅游(sz002033)#股票##炒股##财经##理财##投资#推荐包赢股，盈利对半分成...,2015-02-23,17:41:29,0.0,10.0,...,0,1,0,48,0,0,0,5,0,0
1,d38e9bed5d98110dc2489d0d1cac3c2a,00755196c77936bf44656ada98291c59,0,0,0,想开了就是幸福，想不开就是痛苦…http://t.cn/RLqzYa1,2015-07-13,19:24:50,1.0,0.0,...,0,1,0,48,0,0,0,5,0,0
2,d38e9bed5d98110dc2489d0d1cac3c2a,4fedf3888b1e16592f0e0bdc8b393845,0,0,0,300419浩丰科技#股票##股神##股市##炒股##财经##理财##投资# 股票庄家，要求...,2015-03-14,21:22:57,0.0,14.0,...,0,1,0,48,0,0,0,5,0,0
3,d38e9bed5d98110dc2489d0d1cac3c2a,91be0b8612265aae32725cd4fa80b222,0,0,0,女性被人占便宜从来不生气，是什么心态？http://t.cn/R2n2pRz,2015-06-18,16:25:36,1.0,0.0,...,0,1,0,48,0,0,0,5,0,0
4,d38e9bed5d98110dc2489d0d1cac3c2a,bd2af99ecf1298f5539f0ddfcdd3ed64,0,0,0,网宿科技(sz300017)#股票##炒股##财经##理财##投资#推荐包赢股，盈利对半分成...,2015-02-23,17:35:31,0.0,10.0,...,0,1,0,48,0,0,0,5,0,0


In [10]:
trainstat.head(614809).to_csv("G://DMA_PROJECT//weibo_train1_cpts.csv",sep=',',index=False,encoding='utf-8')
trainstat.tail(614809).to_csv("G://DMA_PROJECT//weibo_train2_cpts.csv",sep=',',index=False,encoding='utf-8')
predictstat.to_csv("G://DMA_PROJECT//weibo_predict_cpts.csv",sep=',',index=False,encoding='utf-8')

In [4]:
df1=pd.read_csv("G://DMA_PROJECT//weibo_train1_cpts.csv")
df2=pd.read_csv("G://DMA_PROJECT//weibo_train2_cpts.csv")
frames=[df1,df2]
train_dataset=pd.concat(frames)
predict_dataset=pd.read_csv("G://DMA_PROJECT//weibo_predict_cpts.csv")

In [5]:
train_dataset['date']=pd.to_datetime(train_dataset['date'],errors='coerce')

In [6]:
train_month=[g for n, g in train_dataset.groupby(pd.Grouper(key='date',freq='M'))]

In [14]:
train_month[0].to_csv("G://DMA_PROJECT//weibo_train_feb_cpts.csv",sep=',',index=False,encoding='utf-8')
train_month[1].to_csv("G://DMA_PROJECT//weibo_train_march_cpts.csv",sep=',',index=False,encoding='utf-8')
train_month[2].to_csv("G://DMA_PROJECT//weibo_train_april_cpts.csv",sep=',',index=False,encoding='utf-8')
train_month[3].to_csv("G://DMA_PROJECT//weibo_train_may_cpts.csv",sep=',',index=False,encoding='utf-8')
train_month[4].to_csv("G://DMA_PROJECT//weibo_train_june_cpts.csv",sep=',',index=False,encoding='utf-8')
train_month[5].to_csv("G://DMA_PROJECT//weibo_train_july_cpts.csv",sep=',',index=False,encoding='utf-8')

In [7]:
train_month[0]=pd.read_csv("G://DMA_PROJECT//weibo_train_feb_cpts.csv")
train_month[1]=pd.read_csv("G://DMA_PROJECT//weibo_train_march_cpts.csv")
train_month[2]=pd.read_csv("G://DMA_PROJECT//weibo_train_april_cpts.csv")
train_month[3]=pd.read_csv("G://DMA_PROJECT//weibo_train_may_cpts.csv")
train_month[4]=pd.read_csv("G://DMA_PROJECT//weibo_train_june_cpts.csv")
train_month[5]=pd.read_csv("G://DMA_PROJECT//weibo_train_july_cpts.csv")

In [8]:
frames1=[train_month[0],train_month[1],train_month[2],train_month[3],train_month[4]]
train=pd.concat(frames1)
predict=train_month[5]

# Model 7 Median,Time: (Hour, Min, Sec), Media, Length, Emoji 
## Only for Forward Count

In [9]:
X_train=train[["content_media_count","content_length","content_emoji_count","hour","min","sec","forward_median"]]
Y_train=train[["forward_count"]]
X_test=predict[["content_media_count","content_length","content_emoji_count","hour","min","sec","forward_median"]]
Y_test=predict[["forward_count"]]

In [10]:
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(1044681, 7) (1044681, 1)
(184937, 7) (184937, 1)


In [11]:
pd.options.mode.use_inf_as_na = True
X_train.fillna(X_train.max(),inplace=True)
X_test.fillna(X_test.max(),inplace=True)

G:\Anaconda\lib\site-packages\pandas\core\generic.py:5430: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [12]:
lm=linear_model.LinearRegression()
model=lm.fit(X_train,Y_train)
pred=lm.predict(X_test)
pred=pred.round()
pred=(np.maximum(pred,0.))

In [13]:
print(pred[0:5])

[[ 1.]
 [ 1.]
 [ 1.]
 [ 8.]
 [ 1.]]


In [14]:
print(model.coef_)

[[ -7.05031752e-01   2.07778896e-02   6.62467259e-02   4.04463406e-02
    6.80481359e-04   9.98897013e-05   1.52380773e+00]]


In [15]:
print(model.intercept_)

[-0.37795983]


In [16]:
np.savetxt("G://DMA_PROJECT//weibo_predict_result8.csv",pred,delimiter=',',header="forward_count,comment_count,like_count",comments="")
result=pd.read_csv("G://DMA_PROJECT//weibo_predict_result8.csv")

In [17]:
train_real_pred=Y_test
train_real_pred['fp']=result['forward_count'].values
train_real_pred=train_real_pred.round()
print ("Score on the training set:{0:.2f}%".format(precision2(train_real_pred.values)*100))

G:\Anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


Score on the training set:45.81%


# Model 7 Median,Time: (Hour, Min, Sec), Media, Length, Emoji 

In [39]:
X_train=train[["content_media_count","content_length","content_emoji_count","hour","min","sec","forward_median","comment_median","like_median"]]
Y_train=train[["forward_count","comment_count","like_count"]]
X_test=predict[["content_media_count","content_length","content_emoji_count","hour","min","sec","forward_median","comment_median","like_median"]]
Y_test=predict[["forward_count","comment_count","like_count"]]

In [40]:
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(1044681, 9) (1044681, 3)
(184937, 9) (184937, 3)


In [41]:
pd.options.mode.use_inf_as_na = True
X_train.fillna(X_train.max(),inplace=True)
X_test.fillna(X_test.max(),inplace=True)

G:\Anaconda\lib\site-packages\pandas\core\generic.py:5430: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [42]:
lm=linear_model.LinearRegression()
model=lm.fit(X_train,Y_train)
pred=lm.predict(X_test)
pred=pred.round()
pred=(np.maximum(pred,0.))

In [43]:
print(pred[0:5])

[[  3.   4.   3.]
 [  3.   4.   3.]
 [  3.   4.   3.]
 [ 13.   7.   8.]
 [  2.   2.   2.]]


In [44]:
print(model.coef_)

[[ -2.63364425e-01   1.91174376e-02   5.54642973e-02   3.76150971e-02
    5.27858190e-04   1.68362209e-03   1.58279691e+00   1.67729136e+00
   -2.93017304e-01]
 [ -6.60961017e-01   2.55079540e-03   1.00504045e-01   1.24809468e-02
    2.35146358e-03   1.20243289e-03   7.17940397e-02   1.55348700e+00
    4.45266999e-03]
 [ -8.70474303e-01   9.23346652e-04   5.98607519e-02   2.34643687e-02
    4.45920014e-05   1.52298289e-03   1.45839466e-01   3.91354761e-01
    1.06591334e+00]]


In [45]:
print(model.intercept_)

[-1.04897282  0.36245024  0.43019058]


In [46]:
np.savetxt("G://DMA_PROJECT//weibo_predict_result9.csv",pred,delimiter=',',header="forward_count,comment_count,like_count",comments="")
result=pd.read_csv("G://DMA_PROJECT//weibo_predict_result9.csv")

In [47]:
train_real_pred = Y_test
train_real_pred['fp']=result['forward_count'].values
train_real_pred['cp']=result['comment_count'].values
train_real_pred['lp']=result['like_count'].values
train_real_pred=train_real_pred.round()
print ("Score on the training set:{0:.2f}%".format(precision(train_real_pred.values)*100))

G:\Anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
G:\Anaconda\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
G:\Anaconda\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.

Score on the training set:22.47%
